## Prolific Data (n =802)

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim
from gensim import corpora
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df802 = pd.read_excel('Recall Prime Text_Tagged_PNAS.xlsx',engine='openpyxl')

In [5]:
df802.columns

Index(['gender', 'age', 'edu', 'yearworked', 'industry', 'employed',
       'supervisor', 'condition', 'text', 'CorrectText'],
      dtype='object')

In [8]:
df802

,gender,age,edu,yearworked,industry,employed,supervisor,condition,text,CorrectText
0,Female,43,complete ba,15,Finance,Yes,Yes,hipower,Once I got the opportunity to rate multiple co...,1
1,Male,78,complete ba,40,Other Miscellaneous,No,NaN,hipower,When I worked in my career I supervised numero...,1
2,Female,63,complete ba,45,Health Science,Yes,No,hipower,I do not think I have ever had the power over ...,0
3,Female,72,complete ba,43,Other Miscellaneous,Yes,No,hipower,I once witnessed a crime and was taken to a lo...,0
4,Male,34,complete ba,16,Business Management and Administration,No,NaN,hipower,I had power over another individual when I was...,1
...,...,...,...,...,...,...,...,...,...,...
797,Female,41,complete ba,20,Education and Training,Yes,No,grocery,I went in by myself while my family stayed in ...,1
798,Female,59,complete ba,36,Education and Training,No,NaN,grocery,At the grocery store we intend to buy only a s...,1
799,Male,27,complete ms,4,Manufacturing,Yes,Yes,grocery,Last week I made my way to the grocery store n...,1
800,Male,34,complete ba,45,"Marketing, Sales, and Service",Yes,Yes,grocery,"It was a Saturday morning, and I decided to vi...",1


### using VADER sentiment analysis

In [113]:
#!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [114]:
def vader_scores(feedbacktext, category):
    analyzer = SentimentIntensityAnalyzer()
    return analyzer.polarity_scores(feedbacktext).get(category)

In [119]:
def calculate_sentiment(text):
    # Run VADER on the text
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(text)
    # Extract the positive score
    vader_pos = scores['pos']
    # Extract the negative score
    vader_neg = scores['neg']
    # Extract the neutral score
    vader_neu = scores['neu']
    # Extract the compound score
    vader_compound = scores['compound']
    # Return compound score
    return vader_compound, vader_pos, vader_neg, vader_neu

In [120]:
df802[['vader_compound', 'vader_pos', 'vader_neg', 'vader_neu']] = df802['text'].apply(lambda x: pd.Series(calculate_sentiment(x)))

In [121]:
df802

,gender,age,edu,yearworked,industry,employed,supervisor,condition,text,CorrectText,polarity,subjectivity,assess,ngrams,vader_sentiment,vader_compound,vader_pos,vader_neg,vader_neu
0,Female,43,complete ba,15,Finance,Yes,Yes,hipower,Once I got the opportunity to rate multiple co...,1,0.201042,0.691667,"[([multiple], 0.0, 0.0, None), ([very, proud],...","[[Once, I, got], [I, got, the], [got, the, opp...","(0.8885, 0.254, 0.121, 0.624)",0.8885,0.254,0.121,0.624
1,Male,78,complete ba,40,Other Miscellaneous,No,NaN,hipower,When I worked in my career I supervised numero...,1,0.250000,0.583333,"[([numerous], 0.0, 0.5, None), ([more], 0.5, 0...","[[When, I, worked], [I, worked, in], [worked, ...","(0.5719, 0.122, 0.036, 0.842)",0.5719,0.122,0.036,0.842
2,Female,63,complete ba,45,Health Science,Yes,No,hipower,I do not think I have ever had the power over ...,0,-0.071429,0.142857,"[([limited], -0.07142857142857142, 0.142857142...","[[I, do, not], [do, not, think], [not, think, ...","(-0.2263, 0.0, 0.04, 0.96)",-0.2263,0.000,0.040,0.960
3,Female,72,complete ba,43,Other Miscellaneous,Yes,No,hipower,I once witnessed a crime and was taken to a lo...,0,0.071684,0.644133,"[([grand], 0.5, 1.0, None), ([mean], -0.3125, ...","[[I, once, witnessed], [once, witnessed, a], [...","(-0.9218, 0.082, 0.148, 0.77)",-0.9218,0.082,0.148,0.770
4,Male,34,complete ba,16,Business Management and Administration,No,NaN,hipower,I had power over another individual when I was...,1,0.210000,0.490000,"[([individual], 0.0, 0.4, None), ([high], 0.16...","[[I, had, power], [had, power, over], [power, ...","(0.7579, 0.206, 0.042, 0.751)",0.7579,0.206,0.042,0.751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797,Female,41,complete ba,20,Education and Training,Yes,No,grocery,I went in by myself while my family stayed in ...,1,0.145556,0.607778,"[([quickly], 0.3333333333333333, 0.5, None), (...","[[I, went, in], [went, in, by], [in, by, mysel...","(0.6124, 0.067, 0.027, 0.905)",0.6124,0.067,0.027,0.905
798,Female,59,complete ba,36,Education and Training,No,NaN,grocery,At the grocery store we intend to buy only a s...,1,0.325000,0.670000,"[([only], 0.0, 1.0, None), ([small], -0.25, 0....","[[At, the, grocery], [the, grocery, store], [g...","(0.989, 0.267, 0.042, 0.69)",0.9890,0.267,0.042,0.690
799,Male,27,complete ms,4,Manufacturing,Yes,Yes,grocery,Last week I made my way to the grocery store n...,1,0.169444,0.532407,"[([last], 0.0, 0.06666666666666667, None), ([n...","[[Last, week, I], [week, I, made], [I, made, m...","(0.8555, 0.066, 0.0, 0.934)",0.8555,0.066,0.000,0.934
800,Male,34,complete ba,45,"Marketing, Sales, and Service",Yes,Yes,grocery,"It was a Saturday morning, and I decided to vi...",1,0.239040,0.477475,"[([local], 0.0, 0.0, None), ([cool], 0.35, 0.6...","[[It, was, a], [was, a, Saturday], [a, Saturda...","(0.9832, 0.145, 0.015, 0.84)",0.9832,0.145,0.015,0.840


In [155]:
#summarize entire dataset
df802.groupby('condition')['vader_compound'].describe()

,count,mean,std,min,25%,50%,75%,max
condition,,,,,,,,
eqpower,176.0,0.563867,0.484761,-0.9657,0.363975,0.78740,0.916900,0.9966
grocery,223.0,0.380097,0.524753,-0.9500,0.000000,0.51060,0.812600,0.9911
hipower,201.0,0.439789,0.561189,-0.9640,0.128000,0.68080,0.897700,0.9927
lowpower,202.0,-0.106420,0.652155,-0.9958,-0.729825,-0.21585,0.487375,0.9815


In [189]:
#summarize only Relevant_PNAS = 1
df802_filt = df802[df802['CorrectText']==1]
df802_filt.groupby('condition')['vader_compound'].describe().round(2)

,count,mean,std,min,25%,50%,75%,max
condition,,,,,,,,
eqpower,147.0,0.62,0.44,-0.78,0.45,0.80,0.93,1.00
grocery,212.0,0.41,0.50,-0.95,0.03,0.58,0.82,0.99
hipower,191.0,0.46,0.54,-0.96,0.19,0.69,0.90,0.99
lowpower,200.0,-0.11,0.65,-1.00,-0.74,-0.22,0.47,0.98


In [83]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats

In [193]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Perform Tukey's HSD post-hoc test
tukey_result = pairwise_tukeyhsd(endog=df802_filt['vader_compound'], groups=df802_filt['condition'], alpha=0.05)

# Convert Tukey's HSD result into a pandas DataFrame
tukey_df802 = pd.DataFrame(data=tukey_result._results_table.data[1:], columns=tukey_result._results_table.data[0])

# Calculate Cohen's d for each comparison
def cohen_d(group1, group2):
    # Get the polarity values for each condition
    data1 = df802_filt[df802_filt['condition'] == group1]['vader_compound']
    data2 = df802_filt[df802_filt['condition'] == group2]['vader_compound']
    
    # Calculate the mean difference
    mean_diff = np.abs(np.mean(data1) - np.mean(data2))
    
    # Calculate pooled standard deviation
    pooled_sd = np.sqrt(((len(data1) - 1) * np.std(data1, ddof=1) ** 2 + (len(data2) - 1) * np.std(data2, ddof=1) ** 2) / (len(data1) + len(data2) - 2))
    
    # Calculate Cohen's d
    d = mean_diff / pooled_sd
    return d

# Apply Cohen's d calculation to each pairwise comparison
cohens_d_values = []
for i in range(len(tukey_df802)):
    group1 = tukey_df802.iloc[i, 0]  # First group in the comparison
    group2 = tukey_df802.iloc[i, 1]  # Second group in the comparison
    d_value = cohen_d(group1, group2)
    cohens_d_values.append(d_value)

# Add Cohen's d values to the Tukey DataFrame
tukey_df802['Cohen_d'] = cohens_d_values


# Display the Tukey test results as a DataFrame
tukey_df802

,group1,group2,meandiff,p-adj,lower,upper,reject,Cohen_d
0,eqpower,grocery,-0.2030,0.0032,-0.3539,-0.0521,True,0.423511
1,eqpower,hipower,-0.1502,0.0598,-0.3044,0.0041,False,0.298558
2,eqpower,lowpower,-0.7279,0.0000,-0.8806,-0.5751,True,1.271073
3,grocery,hipower,0.0529,0.7663,-0.0874,0.1932,False,0.101252
4,grocery,lowpower,-0.5248,0.0000,-0.6634,-0.3862,True,0.906298
5,hipower,lowpower,-0.5777,0.0000,-0.7200,-0.4354,True,0.961484


## Combined Data (n=8400) 

In [57]:
df8400 = pd.read_excel('Power Recall Prime Text Collection Tagged 8400PNAS.xlsx',engine='openpyxl')

In [58]:
df8400.columns

Index(['Year', 'Sample', 'Researcher', 'SourceFile', 'Condition', 'Text',
       'Age', 'Female', 'Country', 'Relevant_Original', 'Relevant_PNAS'],
      dtype='object')

In [59]:
df8400

,Year,Sample,Researcher,SourceFile,Condition,Text,Age,Female,Country,Relevant_Original,Relevant_PNAS
0,2012.0,MTurk,Pitesa & Thau,p1_v3_s1_redo2,HP,A few years ago I was an instructor for teens;...,23.0,1.0,USA,1,1
1,2012.0,MTurk,Pitesa & Thau,p1_v3_s1_redo2,HP,A few years ago I was working as a manager at ...,25.0,0.0,USA,1,1
2,2012.0,MTurk,Pitesa & Thau,p1_v3_s1_redo2,HP,A few years ago on teh job I was assigned the ...,23.0,1.0,USA,1,1
3,2012.0,MTurk,Pitesa & Thau,p1_v3_s1_redo2,HP,"A few years back, before I ventured off to my ...",23.0,1.0,USA,1,1
4,2012.0,MTurk,Pitesa & Thau,p1_v3_s1_redo2,HP,A situation in which i had power over other in...,28.0,1.0,USA,1,1
...,...,...,...,...,...,...,...,...,...,...,...
8395,2020.0,MTurk,"Foulk, De Pater, Schaerer, du Plessis, Lee, & ...",Power+and+Paranoia+R&R+-+Psych+Recall+-+MTurk+...,C (Last Meal),My wife was feeling not well so i made a fruit...,40.0,0.0,USA,0,1
8396,2020.0,MTurk,"Foulk, De Pater, Schaerer, du Plessis, Lee, & ...",Power+and+Paranoia+R&R+-+Psych+Recall+-+MTurk+...,C (Last Meal),I had a frozen pizza last night for dinner. Al...,45.0,1.0,USA,1,1
8397,2020.0,MTurk,"Foulk, De Pater, Schaerer, du Plessis, Lee, & ...",Power+and+Paranoia+R&R+-+Psych+Recall+-+MTurk+...,C (Last Meal),We had a very nice stir fry that my husband co...,58.0,1.0,USA,0,1
8398,2020.0,MTurk,"Foulk, De Pater, Schaerer, du Plessis, Lee, & ...",Power+and+Paranoia+R&R+-+Psych+Recall+-+MTurk+...,C (Last Meal),"I had rice, beans, and meat.I had dinner with ...",35.0,0.0,USA,1,1


### using VADER sentiment

In [167]:
df8400[['vader_compound', 'vader_pos', 'vader_neg', 'vader_neu']] = df8400['Text'].apply(lambda x: pd.Series(calculate_sentiment(x)))

In [169]:
df8400

,Year,Sample,Researcher,SourceFile,Condition,Text,Age,Female,Country,Relevant_Original,Relevant_PNAS,polarity,subjectivity,assess,ngrams,vader_compound,vader_pos,vader_neg,vader_neu
0,2012.0,MTurk,Pitesa & Thau,p1_v3_s1_redo2,HP,A few years ago I was an instructor for teens;...,23.0,1.0,USA,1,1,0.161111,0.477778,"[([few], -0.2, 0.1, None), ([most], 0.5, 0.5, ...","[[A, few, years], [few, years, ago], [years, a...",0.7902,0.114,0.022,0.864
1,2012.0,MTurk,Pitesa & Thau,p1_v3_s1_redo2,HP,A few years ago I was working as a manager at ...,25.0,0.0,USA,1,1,0.146029,0.361829,"[([few], -0.2, 0.1, None), ([few], -0.2, 0.1, ...","[[A, few, years], [few, years, ago], [years, a...",0.9693,0.081,0.017,0.903
2,2012.0,MTurk,Pitesa & Thau,p1_v3_s1_redo2,HP,A few years ago on teh job I was assigned the ...,23.0,1.0,USA,1,1,-0.207143,0.228571,"[([few], -0.2, 0.1, None), ([plain], -0.214285...","[[A, few, years], [few, years, ago], [years, a...",0.3182,0.057,0.031,0.912
3,2012.0,MTurk,Pitesa & Thau,p1_v3_s1_redo2,HP,"A few years back, before I ventured off to my ...",23.0,1.0,USA,1,1,0.097222,0.423611,"[([few], -0.2, 0.1, None), ([back], 0.0, 0.0, ...","[[A, few, years], [few, years, back], [years, ...",0.4559,0.067,0.039,0.894
4,2012.0,MTurk,Pitesa & Thau,p1_v3_s1_redo2,HP,A situation in which i had power over other in...,28.0,1.0,USA,1,1,0.012500,0.379167,"[([other], -0.125, 0.375, None), ([really, slo...","[[A, situation, in], [situation, in, which], [...",0.7351,0.097,0.000,0.903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8395,2020.0,MTurk,"Foulk, De Pater, Schaerer, du Plessis, Lee, & ...",Power+and+Paranoia+R&R+-+Psych+Recall+-+MTurk+...,C (Last Meal),My wife was feeling not well so i made a fruit...,40.0,0.0,USA,0,1,0.000000,0.000000,[],"[[My, wife, was], [wife, was, feeling], [was, ...",-0.0808,0.087,0.105,0.809
8396,2020.0,MTurk,"Foulk, De Pater, Schaerer, du Plessis, Lee, & ...",Power+and+Paranoia+R&R+-+Psych+Recall+-+MTurk+...,C (Last Meal),I had a frozen pizza last night for dinner. Al...,45.0,1.0,USA,1,1,0.095833,0.406944,"[([last], 0.0, 0.06666666666666667, None), ([e...","[[I, had, a], [had, a, frozen], [a, frozen, pi...",0.1710,0.072,0.057,0.871
8397,2020.0,MTurk,"Foulk, De Pater, Schaerer, du Plessis, Lee, & ...",Power+and+Paranoia+R&R+-+Psych+Recall+-+MTurk+...,C (Last Meal),We had a very nice stir fry that my husband co...,58.0,1.0,USA,0,1,0.926667,1.000000,"[([very, nice], 0.78, 1.0, None), ([very, deli...","[[We, had, a], [had, a, very], [a, very, nice]...",0.9017,0.409,0.000,0.591
8398,2020.0,MTurk,"Foulk, De Pater, Schaerer, du Plessis, Lee, & ...",Power+and+Paranoia+R&R+-+Psych+Recall+-+MTurk+...,C (Last Meal),"I had rice, beans, and meat.I had dinner with ...",35.0,0.0,USA,1,1,0.000000,0.000000,[],"[[I, had, rice], [had, rice, beans], [rice, be...",0.0000,0.000,0.000,1.000


In [182]:
#summarize entire dataset
df8400.groupby('Condition')['vader_compound'].describe().round(2)

,count,mean,std,min,25%,50%,75%,max
Condition,,,,,,,,
C (Equal Power),434.0,0.35,0.51,-0.97,0.00,0.44,0.78,0.99
C (Grocery),1440.0,0.26,0.53,-0.98,0.00,0.37,0.74,0.99
C (Last Meal),624.0,0.42,0.48,-0.95,0.00,0.57,0.81,0.97
HP,2442.0,0.37,0.55,-0.99,0.00,0.56,0.83,0.99
LP,3460.0,-0.27,0.60,-0.99,-0.81,-0.46,0.21,1.00


In [187]:
#summarize only Relevant_PNAS = 1
df8400_filt = df8400[df8400['Relevant_PNAS']==1]
df8400_filt.groupby('Condition')['vader_compound'].describe().round(2)

,count,mean,std,min,25%,50%,75%,max
Condition,,,,,,,,
C (Equal Power),341.0,0.41,0.49,-0.95,0.00,0.53,0.82,0.99
C (Grocery),1318.0,0.27,0.54,-0.98,0.00,0.38,0.75,0.99
C (Last Meal),500.0,0.48,0.45,-0.89,0.19,0.64,0.83,0.97
HP,2163.0,0.39,0.56,-0.98,0.00,0.61,0.85,0.99
LP,3010.0,-0.29,0.60,-0.99,-0.82,-0.49,0.16,0.98


In [196]:
#### Perform Tukey's HSD post-hoc test
tukey_result = pairwise_tukeyhsd(endog=df8400_filt['vader_compound'], groups=df8400_filt['Condition'], alpha=0.05)
print(tukey_result)

# Convert Tukey's HSD result into a pandas DataFrame
tukey_df8400 = pd.DataFrame(data=tukey_result._results_table.data[1:], columns=tukey_result._results_table.data[0])

# Calculate Cohen's d for each comparison
def cohen_d(group1, group2):
    # Get the polarity values for each condition
    data1 = df8400_filt[df8400_filt['Condition'] == group1]['vader_compound']
    data2 = df8400_filt[df8400_filt['Condition'] == group2]['vader_compound']
    
    # Calculate the mean difference
    mean_diff = np.abs(np.mean(data1) - np.mean(data2))
    
    # Calculate pooled standard deviation
    pooled_sd = np.sqrt(((len(data1) - 1) * np.std(data1, ddof=1) ** 2 + (len(data2) - 1) * np.std(data2, ddof=1) ** 2) / (len(data1) + len(data2) - 2))
    
    # Calculate Cohen's d
    d = mean_diff / pooled_sd
    return d

# Apply Cohen's d calculation to each pairwise comparison
cohens_d_values = []
for i in range(len(tukey_df8400)):
    group1 = tukey_df8400.iloc[i, 0]  # First group in the comparison
    group2 = tukey_df8400.iloc[i, 1]  # Second group in the comparison
    d_value = cohen_d(group1, group2)
    cohens_d_values.append(d_value)

# Add Cohen's d values to the Tukey DataFrame
tukey_df8400['Cohen_d'] = cohens_d_values


# Display the Tukey test results as a DataFrame
tukey_df8400

        Multiple Comparison of Means - Tukey HSD, FWER=0.05         
     group1         group2    meandiff p-adj   lower   upper  reject
--------------------------------------------------------------------
C (Equal Power)   C (Grocery)  -0.1358 0.0007 -0.2293 -0.0423   True
C (Equal Power) C (Last Meal)   0.0783 0.2771 -0.0297  0.1864  False
C (Equal Power)            HP  -0.0149 0.9913 -0.1046  0.0748  False
C (Equal Power)            LP  -0.6939    0.0 -0.7818 -0.6059   True
    C (Grocery) C (Last Meal)   0.2141    0.0  0.1333  0.2949   True
    C (Grocery)            HP   0.1209    0.0  0.0671  0.1746   True
    C (Grocery)            LP  -0.5581    0.0 -0.6089 -0.5073   True
  C (Last Meal)            HP  -0.0932 0.0077 -0.1696 -0.0169   True
  C (Last Meal)            LP  -0.7722    0.0 -0.8465 -0.6979   True
             HP            LP   -0.679    0.0 -0.7223 -0.6356   True
--------------------------------------------------------------------


,group1,group2,meandiff,p-adj,lower,upper,reject,Cohen_d
0,C (Equal Power),C (Grocery),-0.1358,0.0007,-0.2293,-0.0423,True,0.257363
1,C (Equal Power),C (Last Meal),0.0783,0.2771,-0.0297,0.1864,False,0.167353
2,C (Equal Power),HP,-0.0149,0.9913,-0.1046,0.0748,False,0.027136
3,C (Equal Power),LP,-0.6939,0.0000,-0.7818,-0.6059,True,1.171585
4,C (Grocery),C (Last Meal),0.2141,0.0000,0.1333,0.2949,True,0.415082
5,C (Grocery),HP,0.1209,0.0000,0.0671,0.1746,True,0.219661
6,C (Grocery),LP,-0.5581,0.0000,-0.6089,-0.5073,True,0.956097
7,C (Last Meal),HP,-0.0932,0.0077,-0.1696,-0.0169,True,0.172641
8,C (Last Meal),LP,-0.7722,0.0000,-0.8465,-0.6979,True,1.322102
9,HP,LP,-0.6790,0.0000,-0.7223,-0.6356,True,1.161516
